In [ ]:
import numpy as np
import torch
import torch.nn as nn

from torch.autograd import Variable

import pyro
from pyro.distributions import Normal
from pyro.infer import SVI
from pyro.optim import Adam as Adam_pyro

import quandl
import pendulum
import pandas as pd
import io
import requests
import time

%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
df = pd.read_csv('/content/ADIA Historical Data.csv').fillna(0)
df = df.dropna()
df = df.loc[:,'Price':]


In [ ]:
df.head(5)

In [ ]:

X = df.values
sigma = X.max(axis=0)
X /= sigma

In [ ]:
daily_delta = (df['Price']-df['Open'])/df['Open']
daily_delta = daily_delta.values
X = np.hstack([X, daily_delta[:, None]])

In [ ]:

Y = daily_delta.copy()
Y = df['Price'].values
data_target = Y[1:]
data_feature = X[:-1]
len(data_target)

In [ ]:
plt.plot(data_target)

SIMPLE NN

In [ ]:
from keras.models import Model
from keras.layers import (Input, Dense, Lambda, Conv1D, Reshape, Flatten, GaussianNoise,
                          MaxPooling1D, ActivityRegularization, Activation, GRU)
from keras import backend as K
from keras.regularizers import l2, Regularizer
from tensorflow.keras.optimizers import Adam
import keras
import tensorflow as tf

In [ ]:
num_hidden = 8

In [ ]:
def feature_price(num_features=6, activation='relu', summary=False):
    act = Activation(activation)

    inputs = Input(shape=(num_features,))
    x = Dense(num_hidden)(inputs)
    x = act(x)

    for _ in range(9):
        x = Dense(num_hidden)(x)
        x = act(x)

    outputs = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=inputs, outputs=outputs)
    opt = Adam()

    if summary:
        model.summary()
    return model, lambda: model.compile(optimizer=opt, loss='mse', metrics=['mse'])

In [ ]:
model_nn, build = feature_price(num_features=data_feature.shape[1])
build()

In [ ]:
early = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=10,
    verbose=0,
    mode='auto'
)
tmpfilename = f'Navie_NN.hdf5'
best = keras.callbacks.ModelCheckpoint(
    filepath=tmpfilename,
    
    monitor='val_loss',
    verbose=0,
    save_best_only=True,
    save_weights_only=True,
    mode='min',
    period=1
)

In [ ]:
model_nn.fit(x=data_feature, y=data_target, validation_split=0.2, epochs=5, callbacks=[early, best])

In [ ]:
pred_nn = model_nn.predict(data_feature)
print(pred_nn)

In [ ]:
plt.plot(pred_nn)
plt.plot(data_target, '--')

In [ ]:
weights_nn = model_nn.get_weights()

In [ ]:

std_init = [np.log(np.exp(0.1*np.max(np.abs(weights)))-1) for weights in weights_nn]

BAYESIAN NN

In [ ]:

data_feature_train = torch.Tensor(data_feature[:-200])
data_target_train = torch.Tensor(data_target[:-200])
data = torch.Tensor(np.hstack([data_feature_train, data_target_train[:,None]]))
data_feature_test = data_feature[-200:]
data_target_test = data_target[-200:]

In [ ]:
data = Variable(data)
data_feature_test = Variable(torch.Tensor(data_feature_test))

In [ ]:
pyro.get_param_store().clear()

In [ ]:


class Network(torch.nn.Module):
    def __init__(self, n_feature, n_hidden):
        super(Network, self).__init__()
        self.input = nn.Linear(n_feature, n_hidden)
        for i in range(9):
            setattr(self, f'hidden{i}', nn.Linear(n_hidden, n_hidden))
        self.output = nn.Linear(n_hidden, 1)
        self.activation = nn.ReLU()
    
    def forward(self, x):
        x = self.input(x)
        x = self.activation(x)
        for i in range(9):
            x = getattr(self, f'hidden{i}')(x)
            x = self.activation(x)
        x = self.output(x)
        return nn.Sigmoid()(x)
        return x

In [ ]:
regression_model = Network(data_feature_train.shape[1], num_hidden)


In [ ]:

def model(data):
    data_feature = data[:,:-1]
    data_target = data[:,-1]
    p=data_feature.shape[1]
    
    mu_input = Variable(torch.zeros(num_hidden, p))
    sigma_input = Variable(torch.ones(num_hidden,p))
    bias_mu_input = Variable(torch.zeros(num_hidden))
    bias_sigma_input = Variable(torch.ones(num_hidden))    
    w_prior_input, b_prior_input = Normal(mu_input, sigma_input), Normal(bias_mu_input, bias_sigma_input)
    
    mu_output = Variable(torch.zeros(1, num_hidden))
    sigma_output = Variable(torch.ones(1, num_hidden))
    bias_mu_output = Variable(torch.zeros(1))
    bias_sigma_output = Variable(torch.ones(1))    
    w_prior_output, b_prior_output = Normal(mu_output, sigma_output), Normal(bias_mu_output, bias_sigma_output)
    
    priors = {'input.weight': w_prior_input, 'input.bias': b_prior_input,
              'output.weight': w_prior_output, 'output.bias': b_prior_output}
    
    mu_hidden, sigma_hidden, bias_mu_hidden, bias_sigma_hidden, \
    w_prior_hidden, b_prior_hidden = ({} for _ in range(6))
    for i in range(0,18,2):
        assert weights_nn[i+2].shape == (num_hidden, num_hidden)
        index = int(i/2)
        mu_hidden[index] = Variable(torch.zeros(num_hidden, num_hidden))
        sigma_hidden[index] = Variable(torch.ones(num_hidden, num_hidden))
        bias_mu_hidden[index] = Variable(torch.zeros(num_hidden))
        bias_sigma_hidden[index] = Variable(torch.ones(num_hidden))    
        w_prior_hidden[index] = Normal(mu_hidden[index], sigma_hidden[index])
        b_prior_hidden[index] = Normal(bias_mu_hidden[index], bias_sigma_hidden[index])
        priors[f'hidden{index}.weight'] = w_prior_hidden[index]
        priors[f'hidden{index}.bias'] = b_prior_hidden[index]
    
    lifted_module = pyro.random_module("module", regression_model, priors)
    lifted_reg_model = lifted_module()
    prediction_mean = lifted_reg_model(data_feature).squeeze()
    pyro.sample("obs",
                Normal(prediction_mean, 0.01 * Variable(torch.ones(data.size(0)))),
                obs = data_target.squeeze())

In [ ]:
softplus = nn.Softplus()

In [ ]:
def guide(data):
    data_feature = data[:,:-1]
    p = data_feature.shape[1]
    w_mu_input = Variable(torch.Tensor(weights_nn[0].T), requires_grad=True)
    w_log_sig_input = Variable(std_init[0] * torch.ones(num_hidden, p), requires_grad=True)
    b_mu_input = Variable(torch.Tensor(weights_nn[1]), requires_grad=True)
    b_log_sig_input = Variable(std_init[1] * torch.ones(num_hidden), requires_grad=True)

    mw_param_input = pyro.param("guide_mean_weight_input", w_mu_input)
    sw_param_input = softplus(pyro.param("guide_log_sigma_weight_input", w_log_sig_input))
    mb_param_input = pyro.param("guide_mean_bias_input", b_mu_input)
    sb_param_input = softplus(pyro.param("guide_log_sigma_bias_input", b_log_sig_input))

    w_mu_output = Variable(torch.Tensor(weights_nn[-2].T), requires_grad=True)
    w_log_sig_output = Variable(std_init[-2] * torch.ones(1, num_hidden), requires_grad=True)
    b_mu_output = Variable(torch.Tensor(weights_nn[-1]), requires_grad=True)
    b_log_sig_output = Variable(std_init[-1] * torch.ones(1), requires_grad=True)

    mw_param_output = pyro.param("guide_mean_weight_output", w_mu_output)
    sw_param_output = softplus(pyro.param("guide_log_sigma_weight_output", w_log_sig_output))
    mb_param_output = pyro.param("guide_mean_bias_output", b_mu_output)
    sb_param_output = softplus(pyro.param("guide_log_sigma_bias_output", b_log_sig_output))

    w_dist_input, b_dist_input = Normal(mw_param_input, sw_param_input), Normal(mb_param_input, sb_param_input)
    w_dist_output, b_dist_output = Normal(mw_param_output, sw_param_output), Normal(mb_param_output, sb_param_output)
    
    dists = {'input.weight': w_dist_input, 'input.bias': b_dist_input,
             'output.weight': w_dist_output, 'output.bias': b_dist_output}
    
    w_mu_hidden, w_log_sig_hidden, b_mu_hidden, b_log_sig_hidden, \
    mw_param_hidden, sw_param_hidden, mb_param_hidden, sb_param_hidden, \
    w_dist_hidden, b_dist_hidden = ({} for _ in range(10))
    
    for i in range(0, 18, 2):
        assert weights_nn[i+2].shape == (num_hidden, num_hidden)
        index = int(i/2)
        w_mu_hidden[index] = Variable(torch.Tensor(weights_nn[i+2].T), requires_grad=True)
        w_log_sig_hidden[index] = Variable(std_init[i+2] * torch.ones(num_hidden, num_hidden), requires_grad=True)
        b_mu_hidden[index] = Variable(torch.Tensor(weights_nn[i+3]), requires_grad=True)
        b_log_sig_hidden[index] = Variable(std_init[i+3] * torch.ones(num_hidden), requires_grad=True)

        mw_param_hidden[index] = pyro.param(f'guide_mean_weight_hidden{index}', w_mu_hidden[index])
        sw_param_hidden[index] = softplus(pyro.param(f'guide_log_sigma_weight_hidden{index}', w_log_sig_hidden[index]))
        mb_param_hidden[index] = pyro.param(f'guide_mean_bias_hidden{index}', b_mu_hidden[index])
        sb_param_hidden[index] = softplus(pyro.param(f'guide_log_sigma_bias_hidden{index}', b_log_sig_hidden[index]))
    
        w_dist_hidden[index] = Normal(mw_param_hidden[index], sw_param_hidden[index])
        b_dist_hidden[index] = Normal(mb_param_hidden[index], sb_param_hidden[index])
        dists[f'hidden{index}.weight'] = w_dist_hidden[index]
        dists[f'hidden{index}.bias'] = b_dist_hidden[index]
    
    lifted_module = pyro.random_module("module", regression_model, dists)

    return lifted_module()

In [ ]:

preds = []
for i in range(100):
    sampled_reg_model = guide(data)
    pred = sampled_reg_model(Variable(data_feature_train)).data.numpy().flatten()
    preds.append(pred)

preds = np.array(preds)
mean = np.mean(preds, axis=0)
std = np.std(preds, axis=0) / 10

plt.figure()
x = np.arange(data_target_train.numpy().shape[0])
plt.plot(x, data_target_train.numpy())
plt.plot(x, mean, linestyle = '--')
plt.fill_between(x, mean-std, mean+std, alpha = 0.3, color = 'orange')
plt.show()

In [ ]:
optim = Adam_pyro({"lr": 0.01})
svi = SVI(model, guide, optim, loss="ELBO")

N = len(data_feature_train)

# for j in range(3000):
#     epoch_loss = svi.step(int(data))
#     if j % 100 == 0:
#         print(j, "avg loss {}".format(epoch_loss))

In [ ]:
preds = []
for i in range(100):
    sampled_reg_model = guide(data)
    pred = sampled_reg_model(Variable(data_feature_train)).data.numpy().flatten()
    preds.append(pred)

preds = np.array(preds)
mean = np.mean(preds, axis=0)
std = np.std(preds, axis=0) / 10

plt.figure()
x = np.arange(data_target_train.numpy().shape[0])
plt.plot(x, data_target_train.numpy())
plt.plot(x, mean, linestyle = '--')
plt.fill_between(x, mean-std, mean+std, alpha = 0.3, color = 'orange')
plt.show()